In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250

In [8]:
!pip install -q llama-index-embeddings-langchain llama-index-llms-huggingface

In [10]:
!pip install -Uq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.3 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
    ServiceContext
)
from llama_parse import LlamaParse
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import torch



In [13]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `llm-eng` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate 

In [14]:
import getpass
import os
os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass()

··········


In [15]:
PARSE_TABLES_FIGURES = True

In [16]:
base_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
add_prompt = "Provide tables and images if relevant to the query."
if PARSE_TABLES_FIGURES:
  system_prompt= base_prompt + add_prompt
else:
  system_prompt = base_prompt
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [17]:
system_prompt

'\nYou are a Q&A assistant. Your goal is to answer questions as\naccurately as possible based on the instructions and context provided.\nProvide tables and images if relevant to the query.'

In [18]:
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load model in 4-bit
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute in bfloat16 (or float16)
    bnb_4bit_use_double_quant=True,  # Enable double quantization for efficiency
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"quantization_config": bnb_config}  # Use 4-bit config
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [23]:
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)

In [24]:
PERSIST_DIR = './storage'
if PARSE_TABLES_FIGURES:
  #call the API
  documents = LlamaParse(result_type="markdown").load_data("./data/dsa234.pdf")
  index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
else:
  documents = SimpleDirectoryReader("data").load_data()
  index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

#Persist the index
index.storage_context.persist(persist_dir = PERSIST_DIR)
query_engine = index.as_query_engine(llm=llm)

Started parsing the file under job_id 752b0100-3d3b-4aba-b7ce-3a2035f88013
................

In [ ]:
response=query_engine.query("what is hypertension, also explain the pathophysiology in the form of a table?")
response.responce

Hypertension is a medical condition characterized by consistently elevated blood pressure in the arteries. It is a major risk factor for cardiovascular disease, including heart disease and stroke. The pathophysiology of hypertension involves multiple mechanisms, which are summarized in the following table:

|Pathophysiological Mechanisms of Hypertension|
|---|
|1. Increased cardiac output|
|2. Increased peripheral resistance|
|3. Increased renal resistance|
|4. Increased vasopressin release|
|5. Increased aldosterone release|

Increased cardiac output refers to the heart's ability to pump more blood with each beat, which can lead to increased blood pressure. Increased peripheral resistance refers to the narrowing or constriction of blood vessels, which can also contribute to elevated blood pressure. Increased renal resistance refers to the kidneys' inability to effectively regulate blood pressure, leading to increased blood pressure. Increased vasopressin release refers to the hormone's ability to constrict blood vessels and increase blood pressure. Increased aldosterone release refers to the hormone's ability to increase the body's sodium retention, leading to increased blood volume and blood pressure.

These mechanisms can be influenced by a variety of factors, including genetics, lifestyle choices, and underlying medical conditions. For example, a diet high in sodium can contribute to increased blood pressure, while regular exercise and a healthy diet can help to lower blood pressure. Underlying medical conditions, such as kidney disease or adrenal disorders, can also contribute to hypertension.

It is important to note that hypertension can be a silent killer, as it often does not present with any symptoms until it has caused significant damage to the cardiovascular system. Therefore, it is important to have regular blood pressure checks and to seek medical attention if hypertension is suspected.

I hope this helps! Let me know if you have any further questions.